In [1]:
# Cargamos las librerias que vamos a utilizar

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import ast

In [2]:

#Cargamos el dataset 

d = pd.read_csv('movies_dataset.csv')


C:\Users\hp\AppData\Local\Temp\ipykernel_3732\1674988309.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv('movies_dataset.csv')


In [3]:
# Limpieza de datos
# 1. Se eliminan las columnas que no seran utilizadas

columnas_descartables = ['video','imdb_id','adult','original_title','vote_count','poster_path','homepage']
d = d.drop(columns=columnas_descartables)


In [4]:
# 2.Los valores nulos de los campos revenue y budget se rellena con ceros
# Para el campo revenue

d['revenue'] = d['revenue'].fillna(0)

In [5]:
# Para el campo budget 

d['budget'] = d['budget'].fillna(0)

In [6]:
# 3.Se crea la columna release_year con el formato AAAA-mm-dd
# Se borran los nulos

d = d.dropna(subset=['release_date'])

In [7]:
# Transformando a formato datatime

d['release_date'] = pd.to_datetime(d['release_date'],format='%Y-%m-%d',errors='coerce')

In [8]:
# Comprobando si tiene valores nulos

d['release_date'].isna().sum()

3

In [9]:
# Borrando los valores nulos

d = d.dropna(subset=['release_date'])

In [10]:
# Comprobando si hay valores nulo

d['release_date'].isna().sum()

0

In [11]:
# Se crea la columna release_year con formato datatime

d['release_year'] = d['release_date'].dt.year

In [12]:
d.release_year

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45460    1991
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_year, Length: 45376, dtype: int64

In [13]:
# 4. Crear la columna return(retorno de inversion), dividiendo los campos revenue/budget, cuando no hay datos disponibles 
# para calcular, debera tomar el valor 0

# Pasar de dato type object a float

d['budget'] = pd.to_numeric(d['budget'], downcast='float', errors='coerce')

In [14]:
# Dividiendo los campos revenue/budget

d['return'] = d['revenue'].divide(d['budget']).fillna(0)

In [15]:
# Cambiar los valores infinito por NAN

d['return'].replace([np.inf,-np.inf], np.nan, inplace=True)

In [16]:
# Cambiando los NAN por cero

d['return'].fillna(0, inplace = True)

In [17]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget                 45376 non-null  float64       
 2   genres                 45376 non-null  object        
 3   id                     45376 non-null  object        
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  object        
 7   production_companies   45376 non-null  object        
 8   production_countries   45376 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45376 non-null  float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       45376 non-null  object        
 13  s

In [17]:
# 5. Algunos campos estan anidados deberan ser desanidados para unirlos al dataset
# Para las listas

def fetch_name(obj):
    if isinstance(obj,str) and '{' in obj:
        s = [] 
        for i in ast.literal_eval(obj):
            s.append(i['name']);
        return s

In [18]:
# Para los diccionarios

def fetch_name1(obj):
    if isinstance(obj,str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [19]:
d['genres'] = d['genres'].apply(fetch_name)
d['belongs_to_collection'] = d['belongs_to_collection'].apply(fetch_name1)
d['production_companies'] = d['production_companies'].apply(fetch_name)
d['production_countries'] = d['production_countries'].apply(fetch_name)
d['spoken_languages'] = d['spoken_languages'].apply(fetch_name)

In [20]:
# Dataset con datos limpios

d.head(4)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,1995,12.451801
1,None,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,1995,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,1995,0.000000
3,None,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,1995,5.090760


In [21]:
# Dataset limpio

d.to_csv('d_limpio.csv', index=False)